# Island habitat

This indicator was used in the Southeast Blueprint 2023. Code written by Rua Mordecai and Amy Keister

In [11]:
import os
import arcpy

In [12]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
OutputWorkspace = r"D:\Blueprint\2023\finalIndicatorEdits\ContinentalSoutheastBlueprint2023_FinalIndicators\ContinentalSoutheastBlueprint2023_FinalIndicators\SpatialData"

In [13]:
# define raster used for cell size, extent, and snapping
SERaster= r"D:\Blueprint\2022\Masks\finalRegions_extent\BaseBlueprintExtent2022.tif"

In [14]:
# blueprint marine subregion to deal with noData issues
MarineRaster= r"D:\Blueprint\2023\extent\Marine_Extent_v1.tif"

In [15]:
# define island boundary data
smallIslands = r"D:\Blueprint\2022\Indicators\Islands\boundaries\USGSEsriWCMC_GlobalIslandsv2_SmallIslands.shp"
bigIslands = r"D:\Blueprint\2022\Indicators\Islands\boundaries\USGSEsriWCMC_GlobalIslandsv2_BigIslands.shp"

In [16]:
# define critical habitat input data
criticalHabitatData= r"D:\Blueprint\2022\Indicators\Islands\crithab_all_layers\CRITHAB_POLY.shp"

In [17]:
# define shapefile to clip island file
SEVector = r"D:\Blueprint\2022\Subregions\DraftBaseBlueprintSubRgn.shp"

In [18]:
indicatorName = "IslandHabitat.tif"

## Start analysis

In [19]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutputWorkspace

In [9]:
# Clip islands to Southeast
arcpy.analysis.Clip(smallIslands, SEVector, "smallIslands_SE.shp", None)
arcpy.analysis.Clip(bigIslands, SEVector, "bigIslands_SE.shp", None)

<Result 'D:\\Blueprint\\2022\\Indicators\\Islands\\working\\bigIslands_SE.shp'>

In [10]:
# Merge big and small islands
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.management.Merge("bigIslands_SE.shp;smallIslands_SE.shp", "AllIslands.shp")

In [11]:
# buffer islands to deal with no data. 40km is based on distance needed to close the gap between islands.
# the largest gap is in SE Florida
with arcpy.EnvManager(outputCoordinateSystem=sr):
    arcpy.analysis.Buffer("AllIslands.shp", "AllIslandsBuffer40km.shp", "40 Kilometers", "FULL", "ROUND", "ALL", None, "PLANAR")

In [12]:
# add field to convert to raster
arcpy.management.CalculateField("AllIslandsBuffer40km.shp", "raster", "0", "PYTHON3", '', "SHORT")

<Result 'D:\\Blueprint\\2023\\Indicators\\Islands\\working\\AllIslandsBuffer40km.shp'>

In [13]:
# convert to 30m pixels 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
        arcpy.conversion.FeatureToRaster(in_features="AllIslandsBuffer40km.shp", field="raster", out_raster="zero.tif", cell_size=SERaster)

In [14]:
# Select critical habitat to use for this indicator
out_raster = arcpy.management.SelectLayerByAttribute(criticalHabitatData, "NEW_SELECTION", "comname = 'Piping Plover' Or comname = 'Loggerhead sea turtle' Or comname = 'Cape Sable Thoroughwort' Or comname = 'Florida semaphore Cactus' Or comname = 'Silver rice rat' Or comname = 'Bartrams hairstreak Butterfly'", None)

In [15]:
# Export the shapefile with the species removed
with arcpy.EnvManager(outputCoordinateSystem=sr):
    out_raster2 = arcpy.conversion.FeatureClassToFeatureClass(out_raster, OutputWorkspace, "CriticalHabitatReduced.shp")

In [16]:
# Clip critical habitat to islands
arcpy.analysis.Clip("CriticalHabitatReduced.shp", "AllIslands.shp", "CriticalHabitatIslands.shp", None)

<Result 'D:\\Blueprint\\2023\\Indicators\\Islands\\working\\CriticalHabitatIslands.shp'>

In [17]:
# add a field island critical habitat to convert to raster, give a value of 2
arcpy.management.CalculateField("CriticalHabitatIslands.shp", "raster", "2", "PYTHON3", '', "SHORT")

<Result 'D:\\Blueprint\\2023\\Indicators\\Islands\\working\\CriticalHabitatIslands.shp'>

In [18]:
# add a field to islands to convert to raster
arcpy.management.CalculateField("AllIslands.shp", "raster", "1", "PYTHON3", '', "SHORT")

<Result 'D:\\Blueprint\\2023\\Indicators\\Islands\\working\\AllIslands.shp'>

In [19]:
# convert critical habitat to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    arcpy.conversion.FeatureToRaster(in_features="CriticalHabitatIslands.shp", field="raster", out_raster="CriticalHabitatIslandsNull.tif", cell_size=SERaster)

In [20]:
# change critical habitat raster so there isn't noData on islands
out_raster = arcpy.sa.Con(arcpy.sa.IsNull("CriticalHabitatIslandsNull.tif"),0,"CriticalHabitatIslandsNull.tif"); out_raster.save("CriticalHabitatIslands.tif")

In [14]:
# convert islands to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    arcpy.conversion.FeatureToRaster(in_features="AllIslands.shp", field="raster", out_raster="AllIslandsRas.tif", cell_size=SERaster)

In [15]:
# Combine rasters for indicator categories
out_raster = arcpy.sa.CellStatistics("CriticalHabitatIslands.tif;AllIslandsRas.tif;zero.tif", "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("indic.tif")

In [16]:
# use cell statistics to combine the Marine and base blueprint extent
# This will be used to remvoe deep marine areas outside the blueprint
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    out_raster = arcpy.sa.CellStatistics([MarineRaster, SERaster], "MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("FullExtent.tif")

In [17]:
# remove all areas outside the buffer we made above
out_raster = arcpy.sa.ExtractByMask("indic.tif", "AllIslandsBuffer40km.shp"); out_raster.save("indica.tif")

In [10]:
# clip to full extent of base and marine combined. 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=SERaster, snapRaster=SERaster, cellSize=SERaster):
    out_raster = arcpy.sa.ExtractByMask("indica.tif", "FullExtent.tif"); out_raster.save(indicatorName)

## Finalize indicator

In [11]:
# set code block for next step
codeblock = """
def Reclass(value):
    if value == 2:
        return "2 = Island critical habitat for any of six threatened and endangered species (piping plover, loggerhead sea turtle, Cape Sable thoroughwort, Florida semaphore cactus, silver rice rat, or Bartram’s hairstreak butterfly)"        
    elif value == 1:
        return '1 = Other island area'
    elif value == 0:
        return '0 = Not a coastal island'
"""

In [12]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField(indicatorName, "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\Blueprint\\2023\\Indicators\\Islands\\working\\IslandHabitat.tif'>

In [20]:
# set code block for next step of adding RGB colors
codeblock = """
def Reclass1(Value):
	if Value == 2:
		return 151
	elif Value == 1:
		return 247
	else:
		return 255
		
def Reclass2(Value):
	if Value == 2:
		return 57
	elif Value == 1:
		return 195
	else:
		return 255
		
def Reclass3(Value):
	if Value == 2:
		return 13
	elif Value == 1:
		return 72
	else:
		return 255
		
"""

In [21]:
# calculate Red field
arcpy.management.CalculateField(indicatorName, "Red", "Reclass1(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Green field
arcpy.management.CalculateField(indicatorName, "Green", "Reclass2(!Value!)", "PYTHON3", codeblock, "SHORT")
# calculate Blue field
arcpy.management.CalculateField(indicatorName, "Blue", "Reclass3(!Value!)", "PYTHON3", codeblock, "SHORT")

<Result 'D:\\Blueprint\\2023\\finalIndicatorEdits\\ContinentalSoutheastBlueprint2023_FinalIndicators\\ContinentalSoutheastBlueprint2023_FinalIndicators\\SpatialData\\IslandHabitat.tif'>